In [1]:
import os
import textwrap
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


In [7]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
from datasets import load_dataset
ds = load_dataset(
    "hwaseem04/Aya-testing",
    data_files={"xm3600_captioning": "data/xm3600_captioning-00000-of-00001.parquet"}
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


xm3600_captioning-00000-of-00001.parquet:   0%|          | 0.00/197M [00:00<?, ?B/s]

Generating xm3600_captioning split: 0 examples [00:00, ? examples/s]

In [5]:
ds['xm3600_captioning'][0]

{'sample_id': 1,
 'image_id': '000411001ff7dd4f',
 'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=640x480>,
 'prompt_en': 'Caption the image, short answer.',
 'captions_en': ['a rooster and hens surrounded by green leaves .',
  'a rooster with two hens on a rocky slope with some bushes .'],
 'prompt_ar': 'علّق على الصورة، بإجابة قصيرة.',
 'captions_ar': ['ديك و فرخة علي الأرض', 'ديك وفرخة بحديقة'],
 'prompt_bn': 'ছবির ক্যাপশন দিন, সংক্ষিপ্ত উত্তর।',
 'captions_bn': ['জঙ্গলের মদহে দুইটি বাদামী ও কালো রঙের মুরগি আছে'],
 'prompt_cs': 'Popište obrázek krátkou odpovědí.',
 'captions_cs': ['kohout a slepice v trávě',
  'hnědá slepice a kohout jdoucí po trávě v lese'],
 'prompt_da': 'Giv billedet en billedtekst, kort svar.',
 'captions_da': ['en brun høne og en flerfarvet hane i skovbunden',
  'hane og høne i have'],
 'prompt_de': 'Bildunterschrift für das Bild, kurze Antwort.',
 'captions_de': ['eine henne und ein hanh im steinigem garten im gras .',
  'leicht verschwommene 

In [9]:
from tqdm import tqdm
from PIL import Image
import torch
from transformers import PaliGemmaProcessor, PaliGemmaForConditionalGeneration
from transformers.image_utils import load_image

# Load PaliGemma model and processor
model_id = "google/paligemma2-3b-mix-448"
model = PaliGemmaForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto"
).eval()

processor = PaliGemmaProcessor.from_pretrained(model_id)

# Languages to iterate over
languages = ["en", "bn", "de", "ko", "ru", "zh"]

# Directory to save temp images (needed for this model)
os.makedirs("temp_images_caption", exist_ok=True)

dataset = ds['xm3600_captioning']

for sample in tqdm(dataset, desc="Iterating samples"):
    try:
        image = sample["image"]
        sample_id = sample["sample_id"]

        # Save image if not already saved
        image_path = f"temp_images_caption/{sample_id}.jpg"
        if not os.path.exists(image_path):
            image.save(image_path)

        print(f"\n========== Sample ID: {sample_id} ==========")

        for lang in languages:
            prompt_col = f"prompt_{lang}"
            caption_col = f"captions_{lang}"

            # Ensure expected fields exist in sample
            if prompt_col not in sample or caption_col not in sample:
                print(f"[{lang}] Missing data.")
                continue

            prompt = sample[prompt_col]
            gt_caption = sample[caption_col]

            # Format prompt using PaliGemma's multilingual captioning syntax
            paligemma_prompt = f"caption {lang}"

            # Load image in format expected by processor
            loaded_image = load_image(image_path)

            # Prepare input for the model
            model_inputs = processor(
                text=paligemma_prompt,
                images=loaded_image,
                return_tensors="pt"
            ).to(torch.bfloat16).to(model.device)

            input_len = model_inputs["input_ids"].shape[-1]

            # Generate prediction
            with torch.inference_mode():
                generation = model.generate(
                    **model_inputs,
                    max_new_tokens=100,
                    do_sample=False
                )
                generation = generation[0][input_len:]

            # Decode output tokens to string
            pred_caption = processor.decode(generation, skip_special_tokens=True)

            # Display results
            print(f"\n[{lang.upper()}]")
            print(f"Prompt: {prompt}")
            print(f"GT: {gt_caption}")
            print(f"Pred: {textwrap.fill(pred_caption, width=80)}")

        print("=" * 100)

    except Exception as e:
        print(f"Error processing sample {sample['sample_id']}: {e}")

    break

config.json:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/75.1k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.07G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...


[KO]
Prompt: 이미지에 캡션을 달아주세요. 짧은 답변으로.
GT: ['낙엽 과 잡초 가 많은 산길 의 돌 위에 서 있는 닭 두 마리', '시골 마을 에 풀어 놓고 키우는 닭 암수']
Pred: 일부 닭 은 풀밭 에 서 있습니다.


You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to add `<image>` tokens in the very beginning of your text. For this call, we will infer how many images each text has and add special tokens.



[RU]
Prompt: Подпишите изображение, короткий ответ.
GT: ['Яркий петух и его коричневые куры в парке', 'Курица и петух , бегущие по земле в лесу по траве']
Pred: В поле много кур


Iterating samples:   0%|          | 0/3600 [00:05<?, ?it/s]


[ZH]
Prompt: 为图片添加标题，简短回答。
GT: ['在 山里 中 站着 两只鸡 ， 一只 黄色 另一 只 黑 黄色 ， 它们 俩 站着 看 向 同 一个 方向', '在 野外 绿植 地上 的 公鸡 和 母鸡 近景']
Pred: 几只 鸡 在 草地 上 行走 。
